In [4]:
import sys
sys.path.append('..')

from llm_wrapper import get_chat_llm

llm = get_chat_llm()

✓ Opik tracer initialized for project: customer-support


In [2]:
import os

def get_db_config():
    """
    Get database configuration from environment variables
    """
    return {
        'host': os.getenv('POSTGRES_HOST', 'localhost'),
        'port': os.getenv('POSTGRES_PORT', '6024'),
        'database': os.getenv('POSTGRES_DATABASE', 'langchain'),
        'user': os.getenv('POSTGRES_USER', 'langchain'),
        'password': os.getenv('POSTGRES_PASSWORD', 'langchain')
    }

config = get_db_config()

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

def get_db_engine():
    """
    Create SQLAlchemy engine for PostgreSQL connection
    """
    config = get_db_config()
    
    # Create connection string
    connection_string = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"
    
    # Create engine
    engine = create_engine(connection_string, echo=False)
    
    return engine

engine = get_db_engine()

In [5]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase(engine)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

c:\Users\rushi\miniconda3\envs\aiproject\Lib\site-packages\langchain_community\utilities\sql_database.py:159: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(


In [7]:
toolkit.get_tools()

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001817AE51FA0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001817AE51FA0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001817AE51FA0>),
 QuerySQLCheckerTool(description='Use this tool to 

In [8]:
from langchain_classic import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

['dialect', 'top_k']


In [9]:
system_message = prompt_template.format(dialect="Postgres", top_k=5)


In [10]:
from langchain.agents import create_agent

agent = create_agent(llm, toolkit.get_tools(), system_prompt=system_message)

In [11]:
agent.invoke({
    'messages': ['list all the return order reasons']
})

OPIK: Started logging traces to the "customer-support" project at http://localhost:5173/api/v1/session/redirect/projects/?trace_id=019ad37b-390a-725e-962c-74b8ab6f12cf&path=aHR0cDovL2xvY2FsaG9zdDo1MTczL2FwaQ==.


{'messages': [HumanMessage(content='list all the return order reasons', additional_kwargs={}, response_metadata={}, id='7e3b707f-8771-45bb-b6d7-97f917fe76c8'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 550, 'total_tokens': 563, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-ChVLyJp2Ojmosv4KsARMCDeEmouEC', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 's